In [1]:
import numpy as np
import tensorflow as tf

In [4]:
class ResNet:
    def __init__(self, input_shape=(14, 8, 8), num_residual_blocks=10):
        self.input_shape = input_shape
        self.num_residual_blocks = num_residual_blocks
        self.model = self.build_model()

    def residual_block(self, x, filters):
        y = tf.keras.layers.Conv2D(filters, (3, 3), padding='same')(x)
        y = tf.keras.layers.BatchNormalization()(y)
        y = tf.keras.layers.Activation('relu')(y)
        y = tf.keras.layers.Conv2D(filters, (3, 3), padding='same')(y)
        y = tf.keras.layers.BatchNormalization()(y)
        y = tf.keras.layers.Add()([x, y])
        y = tf.keras.layers.Activation('relu')(y)
        return y

    def build_model(self):
        inputs = tf.keras.layers.Input(shape=self.input_shape)
        x = tf.keras.layers.Conv2D(256, (3, 3), padding='same')(inputs)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)

        for i in range(self.num_residual_blocks):
            x = self.residual_block(x, 256)

        value = tf.keras.layers.Conv2D(1, (1, 1))(x)
        value = tf.keras.layers.BatchNormalization()(value)
        value = tf.keras.layers.Activation('relu')(value)
        value = tf.keras.layers.Flatten()(value)
        value = tf.keras.layers.Dense(256, activation='relu')(value)
        value = tf.keras.layers.Dense(1, activation='linear', name='value')(value)  
        model = tf.keras.models.Model(inputs=inputs, outputs=value)  
        return model

    def compile_model(self, learning_rate=0.001):
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(loss='mse', optimizer=opt)  # Using mean squared error for regression task

    def train(self, X, y, epochs=10, batch_size=1):  # Only one output (y) for centipede value
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size)

    def predict(self, test_data):
        preds = self.model.predict(test_data)
        return preds

In [10]:
data = np.load("eval.npz")
boards = data["boards"]
evaluations = data["evaluations"]

In [11]:
resnet_model = ResNet(input_shape=(14, 8, 8), num_residual_blocks=10)

In [12]:
resnet_model.compile_model()

In [ ]:
resnet_model.train(boards, evaluations, epochs=10, batch_size=32)